In [1]:
import pandas as pd
import numpy as np

In [2]:
initial = pd.read_csv("base_stats.csv").rename({'nomr_combined':'norm_combined'},axis=1)

In [3]:
initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   team           24 non-null     int64  
 1   name           24 non-null     object 
 2   dname          24 non-null     object 
 3   kd             24 non-null     float64
 4   norm_kd        24 non-null     float64
 5   kda            24 non-null     float64
 6   norm_kda       24 non-null     float64
 7   most killed    24 non-null     float64
 8   log_norm_mk    24 non-null     float64
 9   win_perc       24 non-null     float64
 10  combined       24 non-null     float64
 11  norm_combined  24 non-null     float64
dtypes: float64(9), int64(1), object(2)
memory usage: 2.4+ KB


In [4]:
initial.head()

,team,name,dname,kd,norm_kd,kda,norm_kda,most killed,log_norm_mk,win_perc,combined,norm_combined
0,1,iron,IRONKNIGHT INC,1.08,0.67500,1.42,0.692683,35.0,0.617705,0.4427,0.607022,0.746043
1,1,mrspikey,AussieCommander,0.83,0.51875,1.15,0.560976,29.0,0.585032,0.5399,0.551165,0.677393
2,1,suffa,SuFFa,0.63,0.39375,0.93,0.453659,30.0,0.590922,0.3934,0.457933,0.562809
3,2,saxonj23,Saxonj23,1.15,0.71875,1.51,0.736585,35.0,0.617705,0.5292,0.650560,0.799552
4,2,dluith,dluith,0.90,0.56250,1.23,0.600000,32.0,0.602135,0.4802,0.561209,0.689738


In [5]:
initial['assists'] = initial.kda-initial.kd
initial['norm_assists'] = initial.assists/initial.assists.max()
initial['combined'] = (initial.norm_kd+initial.log_norm_mk+initial.win_perc+initial.norm_assists)/4
initial['norm_combined'] = initial.combined/initial.combined.max()

In [6]:
initial.head()

,team,name,dname,kd,norm_kd,kda,norm_kda,most killed,log_norm_mk,win_perc,combined,norm_combined,assists,norm_assists
0,1,iron,IRONKNIGHT INC,1.08,0.67500,1.42,0.692683,35.0,0.617705,0.4427,0.618634,0.765427,0.34,0.739130
1,1,mrspikey,AussieCommander,0.83,0.51875,1.15,0.560976,29.0,0.585032,0.5399,0.584834,0.723606,0.32,0.695652
2,1,suffa,SuFFa,0.63,0.39375,0.93,0.453659,30.0,0.590922,0.3934,0.507562,0.627999,0.30,0.652174
3,2,saxonj23,Saxonj23,1.15,0.71875,1.51,0.736585,35.0,0.617705,0.5292,0.662066,0.819165,0.36,0.782609
4,2,dluith,dluith,0.90,0.56250,1.23,0.600000,32.0,0.602135,0.4802,0.590557,0.730687,0.33,0.717391


In [7]:
bins = [1.0,.99,.9,.8,.7,.6,.5,.4,.3,.2,.01,0.0,np.NINF]
ratings = [800,677,366,240,149,72,0,-72,-142,-240,-366,-677]
bins.sort()
ratings.sort()
initial['elo'] = pd.cut(initial.norm_combined.values,bins=bins,labels=ratings
                  ).astype(int)

In [8]:
quantiles = sorted(initial.norm_combined.quantile([1,.75,.5,.25,0]).values)
quantiles[0]=np.NINF
k_value = [5,10,15,20]
initial['k'] = pd.cut(initial.norm_combined.values,bins=quantiles,labels=k_value
                ).astype(int)

In [9]:
df = initial.merge(initial.groupby('team').agg({"elo":'sum'}
                                              ).reset_index().rename({'elo':'team_elo'},
                                                                     axis=1),
                   on='team', 
                   how='left')
df.head(24)

,team,name,dname,kd,norm_kd,kda,norm_kda,most killed,log_norm_mk,win_perc,combined,norm_combined,assists,norm_assists,elo,k,team_elo
0,1,iron,IRONKNIGHT INC,1.08,0.67500,1.42,0.692683,35.00,0.617705,0.4427,0.618634,0.765427,0.34,0.739130,240,10,629
1,1,mrspikey,AussieCommander,0.83,0.51875,1.15,0.560976,29.00,0.585032,0.5399,0.584834,0.723606,0.32,0.695652,240,10,629
2,1,suffa,SuFFa,0.63,0.39375,0.93,0.453659,30.00,0.590922,0.3934,0.507562,0.627999,0.30,0.652174,149,5,629
3,2,saxonj23,Saxonj23,1.15,0.71875,1.51,0.736585,35.00,0.617705,0.5292,0.662066,0.819165,0.36,0.782609,366,15,755
4,2,dluith,dluith,0.90,0.56250,1.23,0.600000,32.00,0.602135,0.4802,0.590557,0.730687,0.33,0.717391,240,10,755
5,2,kry lxxvi,Kry LXXXVI,0.63,0.39375,0.99,0.482927,21.00,0.528954,0.4686,0.543478,0.672438,0.36,0.782609,149,5,755
6,3,capt_TT,Capt_TT,1.25,0.78125,1.67,0.814634,43.00,0.653469,0.5395,0.721816,0.893092,0.42,0.913043,366,20,846
7,3,divi,Divi8882,0.81,0.50625,1.14,0.556098,34.00,0.612668,0.4610,0.574327,0.710607,0.33,0.717391,240,10,846
8,3,nephew of jimi,taybon,0.76,0.47500,1.13,0.551220,25.00,0.559246,0.4392,0.569448,0.704570,0.37,0.804348,240,5,846
9,4,haydos,Haydos2207,1.23,0.76875,1.59,0.775610,42.00,0.649381,0.5236,0.681085,0.842697,0.36,0.782609,366,15,755


In [10]:
#df.to_csv("full_stats.csv",index=False)

In [11]:
def team_elo_update(df:pd.DataFrame):
    return df.groupby('team').agg({"elo":'sum'}).reset_index().rename({'elo':'team_elo'},axis=1)

In [12]:
team_elo = team_elo_update(df)
team_elo.head()

,team,team_elo
0,1,629
1,2,755
2,3,846
3,4,755
4,5,846


## Round 1-7 Initial

In [13]:
def prob(teamA:int, teamB:int):
    ea = (1/(1+10**(
        (team_elo[team_elo.team==teamB].team_elo.values[0] -
         team_elo[team_elo.team==teamA].team_elo.values[0])/400))
         ).round(3)
    eb = (1/(1+10**(
        (team_elo[team_elo.team==teamA].team_elo.values[0] -
         team_elo[team_elo.team==teamB].team_elo.values[0])/400))
         ).round(3)
    return(ea,eb)

In [14]:
def prob_calc(teamA:int, teamB:int):
    ea = (1/(1+10**(
        (team_elo[team_elo.team==teamB].team_elo.values[0] -
         team_elo[team_elo.team==teamA].team_elo.values[0])/400))
         ).round(3)
    eb = (1/(1+10**(
        (team_elo[team_elo.team==teamA].team_elo.values[0] -
         team_elo[team_elo.team==teamB].team_elo.values[0])/400))
         ).round(3)
    print("Team",teamA,"has a:",ea,"% chance and Team",teamB,"has a:",eb,"% chance of winning")

In [15]:
print("Round 1")
# Team 7 v Team 5
prob_calc(7,5)
# Team 4 v Team 2
prob_calc(4,2)
# Team 1 v Team 3
prob_calc(1,3)
# Team 8 v Team 6
prob_calc(8,6)

Round 1
Team 7 has a: 0.857 % chance and Team 5 has a: 0.143 % chance of winning
Team 4 has a: 0.5 % chance and Team 2 has a: 0.5 % chance of winning
Team 1 has a: 0.223 % chance and Team 3 has a: 0.777 % chance of winning
Team 8 has a: 0.67 % chance and Team 6 has a: 0.33 % chance of winning


In [16]:
print("Round 2")
# Team 3 v Team 4
r2t3 = (1/(1+10**((team_elo[team_elo.team==4].team_elo.values[0]-team_elo[team_elo.team==3].team_elo.values[0])/400))).round(3)
r2t4 = (1/(1+10**((team_elo[team_elo.team==3].team_elo.values[0]-team_elo[team_elo.team==4].team_elo.values[0])/400))).round(3)
print("Team 3 has a:",r2t3,"% chance and Team 4 has a:",r2t4,"% chance of winning")
# Team 2 v Team 8
r2t2 = (1/(1+10**((team_elo[team_elo.team==8].team_elo.values[0]-team_elo[team_elo.team==2].team_elo.values[0])/400))).round(3)
r2t8 = (1/(1+10**((team_elo[team_elo.team==2].team_elo.values[0]-team_elo[team_elo.team==8].team_elo.values[0])/400))).round(3)
print("Team 2 has a:",r2t2,"% chance and Team 8 has a:",r2t8,"% chance of winning")
# Team 5 v Team 1
r2t5 = (1/(1+10**((team_elo[team_elo.team==1].team_elo.values[0]-team_elo[team_elo.team==5].team_elo.values[0])/400))).round(3)
r2t1 = (1/(1+10**((team_elo[team_elo.team==5].team_elo.values[0]-team_elo[team_elo.team==1].team_elo.values[0])/400))).round(3)
print("Team 5 has a:",r2t5,"% chance and Team 1 has a:",r2t1,"% chance of winning")
# Team 6 v Team 7
r2t6 = (1/(1+10**((team_elo[team_elo.team==7].team_elo.values[0]-team_elo[team_elo.team==6].team_elo.values[0])/400))).round(3)
r2t7 = (1/(1+10**((team_elo[team_elo.team==6].team_elo.values[0]-team_elo[team_elo.team==7].team_elo.values[0])/400))).round(3)
print("Team 6 has a:",r2t6,"% chance and Team 7 has a:",r2t7,"% chance of winning")

Round 2
Team 3 has a: 0.628 % chance and Team 4 has a: 0.372 % chance of winning
Team 2 has a: 0.038 % chance and Team 8 has a: 0.962 % chance of winning
Team 5 has a: 0.777 % chance and Team 1 has a: 0.223 % chance of winning
Team 6 has a: 0.55 % chance and Team 7 has a: 0.45 % chance of winning


In [17]:
print("Round 3")
# Team 4 v Team 5
r3t4 = (1/(1+10**((team_elo[team_elo.team==5].team_elo.values[0]-team_elo[team_elo.team==4].team_elo.values[0])/400))).round(3)
r3t5 = (1/(1+10**((team_elo[team_elo.team==4].team_elo.values[0]-team_elo[team_elo.team==5].team_elo.values[0])/400))).round(3)
print("Team 4 has a:",r3t4,"% chance and Team 5 has a:",r3t5,"% chance of winning")
# Team 1 v Team 6
r3t1 = (1/(1+10**((team_elo[team_elo.team==6].team_elo.values[0]-team_elo[team_elo.team==1].team_elo.values[0])/400))).round(3)
r3t6 = (1/(1+10**((team_elo[team_elo.team==1].team_elo.values[0]-team_elo[team_elo.team==6].team_elo.values[0])/400))).round(3)
print("Team 1 has a:",r3t1,"% chance and Team 6 has a:",r3t6,"% chance of winning")
# Team 8 v Team 7
r3t8 = (1/(1+10**((team_elo[team_elo.team==7].team_elo.values[0]-team_elo[team_elo.team==8].team_elo.values[0])/400))).round(3)
r3t7 = (1/(1+10**((team_elo[team_elo.team==8].team_elo.values[0]-team_elo[team_elo.team==7].team_elo.values[0])/400))).round(3)
print("Team 8 has a:",r3t8,"% chance and Team 7 has a:",r3t7,"% chance of winning")
# Team 2 v Team 3
r3t2 = (1/(1+10**((team_elo[team_elo.team==3].team_elo.values[0]-team_elo[team_elo.team==2].team_elo.values[0])/400))).round(3)
r3t3 = (1/(1+10**((team_elo[team_elo.team==2].team_elo.values[0]-team_elo[team_elo.team==3].team_elo.values[0])/400))).round(3)
print("Team 2 has a:",r3t2,"% chance and Team 3 has a:",r3t3,"% chance of winning")

Round 3
Team 4 has a: 0.372 % chance and Team 5 has a: 0.628 % chance of winning
Team 1 has a: 0.038 % chance and Team 6 has a: 0.962 % chance of winning
Team 8 has a: 0.713 % chance and Team 7 has a: 0.287 % chance of winning
Team 2 has a: 0.372 % chance and Team 3 has a: 0.628 % chance of winning


In [18]:
# Round 4
# Team 7 v Team 1
r4t7 = (1/(1+10**((team_elo[team_elo.team==1].team_elo.values[0]-team_elo[team_elo.team==7].team_elo.values[0])/400))).round(3)
r4t1 = (1/(1+10**((team_elo[team_elo.team==7].team_elo.values[0]-team_elo[team_elo.team==1].team_elo.values[0])/400))).round(3)
print("Team 7 has a:",r4t7,"% chance and Team 1 has a:",r4t1,"% chance of winning")
# Team 5 v Team 2
r4t5 = (1/(1+10**((team_elo[team_elo.team==2].team_elo.values[0]-team_elo[team_elo.team==5].team_elo.values[0])/400))).round(3)
r4t2 = (1/(1+10**((team_elo[team_elo.team==5].team_elo.values[0]-team_elo[team_elo.team==2].team_elo.values[0])/400))).round(3)
print("Team 5 has a:",r4t5,"% chance and Team 2 has a:",r4t2,"% chance of winning")
# Team 6 v Team 4
r4t6 = (1/(1+10**((team_elo[team_elo.team==4].team_elo.values[0]-team_elo[team_elo.team==6].team_elo.values[0])/400))).round(3)
r4t4 = (1/(1+10**((team_elo[team_elo.team==6].team_elo.values[0]-team_elo[team_elo.team==4].team_elo.values[0])/400))).round(3)
print("Team 6 has a:",r4t6,"% chance and Team 4 has a:",r4t4,"% chance of winning")
# Team 3 v Team 8
r4t3 = (1/(1+10**((team_elo[team_elo.team==8].team_elo.values[0]-team_elo[team_elo.team==3].team_elo.values[0])/400))).round(3)
r4t8 = (1/(1+10**((team_elo[team_elo.team==3].team_elo.values[0]-team_elo[team_elo.team==8].team_elo.values[0])/400))).round(3)
print("Team 3 has a:",r4t3,"% chance and Team 8 has a:",r4t8,"% chance of winning")

Team 7 has a: 0.954 % chance and Team 1 has a: 0.046 % chance of winning
Team 5 has a: 0.628 % chance and Team 2 has a: 0.372 % chance of winning
Team 6 has a: 0.925 % chance and Team 4 has a: 0.075 % chance of winning
Team 3 has a: 0.063 % chance and Team 8 has a: 0.937 % chance of winning


In [19]:
print("Round 5")
# Team 3 v Team 5
prob_calc(3,5)
# Team 4 v Team 7
prob_calc(4,7)
# Team 2 v Team 6
prob_calc(2,6)
# Team 8 v Team 1
prob_calc(8,1)

Round 5
Team 3 has a: 0.5 % chance and Team 5 has a: 0.5 % chance of winning
Team 4 has a: 0.09 % chance and Team 7 has a: 0.91 % chance of winning
Team 2 has a: 0.075 % chance and Team 6 has a: 0.925 % chance of winning
Team 8 has a: 0.981 % chance and Team 1 has a: 0.019 % chance of winning


In [20]:
print("Round 6")
# Team 7 v Team 2
prob_calc(7,2)
# Team 6 v Team 3
prob_calc(6,3)
# Team 5 v Team 8
prob_calc(5,8)
# Team 1 v Team 4
prob_calc(1,4)

Round 6
Team 7 has a: 0.91 % chance and Team 2 has a: 0.09 % chance of winning
Team 6 has a: 0.88 % chance and Team 3 has a: 0.12 % chance of winning
Team 5 has a: 0.063 % chance and Team 8 has a: 0.937 % chance of winning
Team 1 has a: 0.326 % chance and Team 4 has a: 0.674 % chance of winning


In [21]:
print("Round 7")
# Team 2 v Team 1
prob_calc(2,1)
# Team 8 v Team 4
prob_calc(8,4)
# Team 3 v Team 7
prob_calc(3,7)
# Team 5 v Team 6
prob_calc(5,6)

Round 7
Team 2 has a: 0.674 % chance and Team 1 has a: 0.326 % chance of winning
Team 8 has a: 0.962 % chance and Team 4 has a: 0.038 % chance of winning
Team 3 has a: 0.143 % chance and Team 7 has a: 0.857 % chance of winning
Team 5 has a: 0.12 % chance and Team 6 has a: 0.88 % chance of winning


In [22]:
def elo_update(df:pd.DataFrame,teamA:int,teamB:int,result:int):
    """
    Takes a team number as a winner and provides the new elo scores for that team
    """
    ea,eb = prob(teamA,teamB)
    if teamA == result:
        for member in df[df.team==teamA].index:
            df.iloc[member,14]=df.iloc[member,14]+df.iloc[member,15]*(1-ea)
        for member in df[df.team==teamB].index:
            df.iloc[member,14]=df.iloc[member,14]+df.iloc[member,15]*(0-eb)
    else:
        for member in df[df.team==teamA].index:
            df.iloc[member,14]=df.iloc[member,14]+df.iloc[member,15]*(0-ea)
        for member in df[df.team==teamB].index:
            df.iloc[member,14]=df.iloc[member,14]+df.iloc[member,15]*(1-eb)
    return df.merge(df.groupby('team').agg({"elo":'sum'}).reset_index().rename({'elo':'team_elo'},axis=1),on='team', how='left')

## Round 2 Updated

In [23]:
print("Round 2")
# Team 3 v Team 4
prob_calc(3,4)
# Team 2 v Team 8
prob_calc(2,8)
# Team 2 v Team 6
prob_calc(5,1)
# Team 8 v Team 1
prob_calc(6,7)

Round 2
Team 3 has a: 0.628 % chance and Team 4 has a: 0.372 % chance of winning
Team 2 has a: 0.038 % chance and Team 8 has a: 0.962 % chance of winning
Team 5 has a: 0.777 % chance and Team 1 has a: 0.223 % chance of winning
Team 6 has a: 0.55 % chance and Team 7 has a: 0.45 % chance of winning


In [24]:
round2 = elo_update(initial,7,5,7)
round2 = elo_update(round2,4,2,4)
#round2 = elo_update(round2,1,3,x)
#round2 = elo_update(round2,8,6,x)

In [25]:
team_elo = team_elo_update(round2)

In [26]:
print("Round 2")
# Team 3 v Team 4
prob_calc(3,4)
# Team 2 v Team 8
prob_calc(2,8)
# Team 2 v Team 6
prob_calc(5,1)
# Team 8 v Team 1
prob_calc(6,7)

Round 2
Team 3 has a: 0.608 % chance and Team 4 has a: 0.392 % chance of winning
Team 2 has a: 0.035 % chance and Team 8 has a: 0.965 % chance of winning
Team 5 has a: 0.77 % chance and Team 1 has a: 0.23 % chance of winning
Team 6 has a: 0.541 % chance and Team 7 has a: 0.459 % chance of winning
